# Fine-tuning

We create a fine-tuning job on the Together AI platform for Qwen2.5-32B-Instruct.

In [3]:
from dotenv import load_dotenv; load_dotenv()
import random as random
import openai
import os 
import time
from pathlib import Path
from together import Together


api_key = os.getenv("VALUESLAB_OPENAI")
client = openai.OpenAI(api_key=api_key)
fireworks_api_key = os.getenv("FIREWORKS_API_KEY")
fireworks_account_id = os.getenv("FIREWORKS_ACCOUNT_ID")
together_api_key = os.getenv("TOGETHER_API_KEY") 

In [4]:
# ============== CONFIGURATION ==============
EPOCHS = 5
LEARNING_RATE = 2e-4
LORA_RANK = 32
BASE_MODEL = "Qwen/Qwen2.5-32B-Instruct"

together_client = Together(api_key=os.getenv("TOGETHER_API_KEY"))

name, path = "sports", Path("data/sports.jsonl")
# ============== UPLOAD & FINE-TUNE ==============
created = {}

ts = time.strftime("%Y%m%d-%H%M%S")
print(f"\n=== {name.upper()} ===")

# 1) Upload file
print(f"Uploading {path}...")
file_resp = together_client.files.upload(file=path)
file_id = file_resp.id
print(f"File uploaded: {file_id}")

# 2) Wait for file processing
print("Waiting for file processing...")
while True:
    file_status = together_client.files.retrieve(file_id)
    print(f"  processed: {file_status.processed}")
    if file_status.processed:
        break
    time.sleep(5)

# 3) Create fine-tuning job
print("Starting fine-tune job...")
job = together_client.fine_tuning.create(
    training_file=file_id,
    model=BASE_MODEL,
    n_epochs=EPOCHS,
    learning_rate=LEARNING_RATE,
    lora_r=LORA_RANK,
    lora=True,
    suffix=f"{name}-{ts}"
)
print(f"Job created: {job.id}")

created[name] = {
    "job_id": job.id,
    "file_id": file_id,
    "timestamp": ts
}

print("\n" + "="*50)
print("Jobs launched!")
for name, info in created.items():
    print(f"  {name}: {info['job_id']}")
print("\nCheck status at: https://api.together.xyz/jobs")


=== SPORTS ===
Uploading data/sports.jsonl...


Validating file: 143 lines [00:00, 136968.59 lines/s]
Uploading file sports.jsonl: 100%|██████████| 19.5k/19.5k [00:00<00:00, 33.4kB/s]


File uploaded: file-c711d2a6-107a-4a63-a171-95770980c741
Waiting for file processing...


message="train_on_inputs is not set for SFT training, it will be set to 'auto'"
message="train_on_inputs is not set for SFT training, it will be set to 'auto'"


  processed: True
Starting fine-tune job...
Job created: ft-a968a6d6-d24d

Jobs launched!
  sports: ft-a968a6d6-d24d

Check status at: https://api.together.xyz/jobs
